In [1]:
import tensorflow as tf
import pandas as pd

In [2]:
rnn_num_units=128
time_step=1
class_num=33
inputs = tf.placeholder(shape=[None, time_step,74], dtype=tf.float32, name='inputs')
# is_training = tf.placeholder(dtype=tf.bool, name='is_training')
label = tf.placeholder(shape=[None, class_num], dtype=tf.float32)
label_ = tf.placeholder(shape=[None, class_num], dtype=tf.float32)

# repay_amt=tf.placeholder(tf.float32,[None,],name='repay_amt')
# due_amt=tf.placeholder(tf.float32,[None,],name='due_amt')

seq_len = tf.ones(tf.shape(inputs)[0], tf.int32) * time_step
shape = tf.shape(inputs)



#第一层
# with tf.variable_scope(None,'first_2LSTM'):
#     cell = tf.nn.rnn_cell.LSTMCell(num_units=rnn_num_units, num_proj=None)
#     outputs, _ = tf.nn.bidirectional_dynamic_rnn(cell,
#                                              cell,
#                                              inputs,
#                                              sequence_length=seq_len,
#                                              dtype=tf.float32)

#     outputs = tf.concat(outputs, 2)
#     outputs = tf.reshape(outputs, [-1, rnn_num_units * 2])

#     outputs = tf.layers.dense(outputs, 256, activation=tf.nn.leaky_relu)


#     outputs = tf.reshape(outputs, [shape[0], -1, 256])  # [batch,time_step,feature]


# #第二层
# with tf.variable_scope(None,'second_2LSTM'):
#     cell = tf.nn.rnn_cell.LSTMCell(num_units=rnn_num_units, num_proj=None)
#     outputs, _ = tf.nn.bidirectional_dynamic_rnn(cell,
#                                              cell,
#                                              outputs,
#                                              sequence_length=seq_len,
#                                              dtype=tf.float32)

#     outputs = tf.concat(outputs, 2)
#     outputs = tf.reshape(outputs, [-1, rnn_num_units * 2])

#     outputs = tf.layers.dense(outputs, class_num, activation=tf.nn.leaky_relu)

#     shape = tf.shape(inputs)
#     outputs = tf.reshape(outputs, [shape[0], -1, class_num])  # [batch,time_step,feature]

# 最后一层
with tf.variable_scope(None,'third_2LSTM'):
    cell = tf.nn.rnn_cell.LSTMCell(num_units=rnn_num_units, num_proj=None)
    outputs, _ = tf.nn.bidirectional_dynamic_rnn(cell,
                                             cell,
                                             inputs,#outputs,
                                             sequence_length=seq_len,
                                             dtype=tf.float32)
    outputs = tf.concat(outputs, 2)
    outputs = tf.reshape(outputs, [-1, rnn_num_units * 2])

    logits = tf.layers.dense(outputs, class_num, activation=None)


#计算损失
y_pred = tf.nn.softmax(logits)
y_pred = tf.clip_by_value(y_pred, 1e-10, 1.0 - 1e-10)
loss = -label * ((1 - y_pred) ** 2.0) * tf.log(y_pred)
loss = tf.reduce_mean(loss)

# rmse=tf.sqrt(tf.reduce_mean(tf.pow(tf.reduce_sum(y_pred*label_,-1)*due_amt-repay_amt,2)))

correct_pred = tf.equal(tf.argmax(y_pred,1),tf.argmax(label,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))


global_step = tf.Variable(0, trainable=False)
lr = tf.train.exponential_decay(0.1,global_step,5000,0.95,staircase=True)  # staircase=True,globstep/decaystep=整数，代表lr突变的，阶梯状
optimizer = tf.train.AdamOptimizer(learning_rate=lr)

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train_op = optimizer.minimize(loss, global_step=global_step)

# tf.summary.scalar('total_loss', loss)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dense instead.


In [3]:
training_epochs = 20
batch_size = 256
file='./train_set_onehot.csv'

In [4]:
saver=tf.train.Saver(max_to_keep=1)
init = tf.global_variables_initializer()

In [6]:
with tf.Session() as sess:
    sess.run(init)
#     summary_writer=tf.summary.FileWriter('../tensorboard_summary/rnn_summary',sess.graph)
#     model_ckpt = tf.train.latest_checkpoint('../save_models/')
#     print(model_ckpt)
#     saver.restore(sess,model_ckpt)

    for epoch in range(training_epochs):
        batches = pd.read_csv(file,chunksize=batch_size)
        total_batch = 0
        for batch in batches:
            total_batch +=1
#             print(batch.iloc[:,:].info())
            input_features = batch.iloc[:,1:75]
            input_features = np.expand_dims(input_features,1)
            labels = batch.iloc[:,75:].values
#             print(labels.shape)
            labels_ = labels.copy()
#             labels_[:,32] = 0.
#             print(labels_.shape)
            repay_amt_ = batch.iloc[:,0].values
            due_amt_ = batch.iloc[:,1].values
            
            _,cost_,acc,global_step_,learning_rate=sess.run(
                            [train_op,loss,accuracy,global_step,lr],
                            feed_dict= { inputs:input_features, label:labels})#, label_:labels_})#,
                                        #repay_amt:repay_amt_, due_amt:due_amt_})
#             summary_writer.add_summary(merge_summary,global_step_)
            if global_step_%100==0:
                print('[epoch %s][step %s]' % (epoch,global_step_),
                     'loss:%.6f' % cost_,
#                      'l2_regular:%.6f' % l2_regular_,
#                      'rmse:%.6f' % rmse_,
#                      'total_loss:%.6f' % total_loss_,
                     'acc:%.6f' % acc,
                     'lr:%.6f' % learning_rate)
            
            if global_step_%1000==0:
                saver.save(sess,"../save_models/lstm.ckpt",global_step=global_step_)
        print('--------------epoch %s finished —> total batch:%s---------------' % (epoch,total_batch))
    print('finished!')

InternalError: Blas GEMM launch failed : a.shape=(256, 202), b.shape=(202, 512), m=256, n=512, k=202
	 [[node third_2LSTM/bidirectional_rnn/fw/fw/while/lstm_cell/MatMul_1 (defined at <ipython-input-2-dfbe2801c0d4>:59) ]]
	 [[node Adam/update (defined at <ipython-input-2-dfbe2801c0d4>:84) ]]

Caused by op 'third_2LSTM/bidirectional_rnn/fw/fw/while/lstm_cell/MatMul_1', defined at:
  File "F:\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "F:\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "F:\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "F:\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "F:\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "F:\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 148, in start
    self.